In [163]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras import Model
from tensorflow import keras

import numpy as np

In [164]:
# Fungsi untuk membuat model kontrastif
def create_contrastive_model(input_shape, embedding_size=64):
    input_a = tf.keras.Input(shape=input_shape)
    input_b = tf.keras.Input(shape=input_shape)

    # Bagian jaringan konvolusi bersama
    shared_conv = models.Sequential([
        layers.Conv2D(32, (5, 5), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (5, 5), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (5, 5), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(embedding_size, activation='relu')
    ])

    # Menghasilkan representasi vektor untuk setiap input
    output_a = shared_conv(input_a)
    output_b = shared_conv(input_b)

    # Layer untuk menghitung jarak Euclidean antara dua vektor
    distance = layers.Lambda(lambda embeddings: tf.norm(embeddings[0] - embeddings[1], axis=1, keepdims=True), name='distance')([output_a, output_b])

    # Membangun model
    model = Model(inputs=[input_a, input_b], outputs=distance)
    return model

In [165]:
input_shape = (160, 160, 1)  # Sesuaikan dengan ukuran gambar yang digunakan
contrastive_model = create_contrastive_model(input_shape)

In [166]:
def contrastive_loss(y_true, distance):
    margin = 1.0
    return tf.reduce_mean(y_true * tf.square(distance) + (1 - y_true) * tf.square(tf.maximum(margin - distance, 0)))

In [167]:

contrastive_model.compile(optimizer='adam', loss=contrastive_loss)

In [168]:
contrastive_model.summary()

Model: "model_41"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_53 (InputLayer)       [(None, 160, 160, 1)]        0         []                            
                                                                                                  
 input_54 (InputLayer)       [(None, 160, 160, 1)]        0         []                            
                                                                                                  
 sequential_11 (Sequential)  (None, 64)                   2354240   ['input_53[0][0]',            
                                                                     'input_54[0][0]']            
                                                                                                  
 distance (Lambda)           (None, 1)                    0         ['sequential_11[0][0]',

In [169]:
contrastive_model.save('facematching.h5')

In [171]:
from keras.models import load_model
custom_objects = {'contrastive_loss': contrastive_loss}
model = load_model('facematching.h5', custom_objects=custom_objects)

In [136]:
import cv2
import os
from mtcnn.mtcnn import MTCNN
import matplotlib.pyplot as plt

In [137]:
class FACELOADING:
    def __init__(self):
        self.target_size = (160, 160)  # Adjusted target size for the grayscale images
        self.detector = MTCNN()

    def extract_face(self, img):
        # Convert BGR to RGB
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Detect faces
        results = self.detector.detect_faces(img_rgb)
        
        # Check if a face is detected
        if results:
            # Get the first face (assuming there is only one)
            x, y, w, h = results[0]['box']
            
            # Crop the face region
            cropped_face = img[y:y+h, x:x+w]
            
            # Convert to grayscale and resize
            cropped_face = cv2.cvtColor(cropped_face, cv2.COLOR_BGR2GRAY)
            cropped_face = cv2.resize(cropped_face, self.target_size)
            
            return cropped_face
        else:
            # Return None if no face is detected
            return None

    def load_faces(self, dir):
        faces = []
        for im_name in os.listdir(dir):
            try:
                path = os.path.join(dir, im_name)
                img = cv2.imread(path)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                single_face = self.extract_face(img)
                if single_face is not None:
                    faces.append(single_face)
            except Exception as e:
                pass
        return faces


In [138]:
loader = FACELOADING()

In [172]:
base_images_dir = 'data/base_image/ojiie'
base_images = loader.load_faces(base_images_dir)
base_images = np.array(base_images)

1/1 [==============================] - 0s 52ms/step


In [173]:
print(f"Shape of base_images: {base_images.shape}")

Shape of base_images: (3, 160, 160)


In [187]:
verif_image_dir = 'data/verif_image/aaang'
verif_image = loader.load_faces(verif_image_dir)
verif_image = np.array(verif_image)

1/1 [==============================] - 0s 35ms/step


In [188]:
print(f"Shape of verif_images: {verif_image.shape}")

Shape of verif_images: (3, 160, 160)


In [189]:


# Perform prediction
prediction = model.predict([base_images, verif_image])

# Display the prediction
print(f'Prediction: {prediction[0][0]}')


1/1 [==============================] - 0s 135ms/step
Prediction: 33.29273223876953
